In [ ]:
import os.path as op

import matplotlib.pyplot as plt

import mne
from mne_bids import (BIDSPath, write_raw_bids, read_raw_bids, 
                      write_meg_calibration, write_meg_crosstalk, 
                      get_anat_landmarks, write_anat)

In [ ]:
# fill these out
subject = '01'  # subject code in mTBI project
session = '01'  # data collection session within each run
task = 'SpAtt'  # name of the task

# specify specific file names
MEG_data_root = r'Z:\Projects\FLUX\MEG_data'  # RDS folder for MEG data
bids_root = op.join(MEG_data_root, 'BIDS-data')
file_extension = '.fif'

# Define the fine calibration and cross-talk compensation files for BIDS
maxfilter_folder = op.join(MEG_data_root, 'MaxFilter')
crosstalk_file = op.join(maxfilter_folder, 'CT_sparse_SA.fif')  #'reduces interference' 
                                                                #'between Elekta's co-located' 
                                                                #'magnetometer and'
                                                                #'paired gradiometer sensor units'
calibration_file = op.join(maxfilter_folder, 'sss_cal_SA.dat')  #'encodes site-specific'
                                                                #'information about sensor' 
                                                                #'orientations and calibration'
events_id = {'block_start':1, 'trial_start':3, 'trial_end':4,
             'target_onset':5, 'ITI':10, 'cue_left':21, 'cue_right':22,
             'unknown_1':55, 'unknown_2':56}    

In [ ]:
# Read the events from stim channel
for run in range(1,3):
    file_name = f'training_raw-{run}'
    raw_fname = op.join(MEG_data_root, file_name + file_extension) 
    filename_events = op.join(MEG_data_root, file_name + '-eve' + file_extension)


    # read raw and define the stim channel
    raw = mne.io.read_raw_fif(raw_fname, allow_maxshield=True, 
                              verbose=True, preload=False)
    stim_channel = 'STI101'


    events = mne.find_events(raw, stim_channel=stim_channel, min_duration=0.001001,
                             consecutive=False, mask=65280,
                             mask_type='not_and')  #' mask removes triggers associated
                                                   # with response box channel 
                                                   # (not the response triggers)'
   
    # Save the events in a dedicted FIF-file: 
    mne.write_events(filename_events, events, overwrite=True)
    
    # Visualise a part of the events-array
    plt.figure()
    plt.stem(events[:,0], events[:,2])
    plt.xlim(min(events[:,0]), min(events[:,0])+100000)
    plt.title('events')
    plt.xlabel('sample')
    plt.ylabel('Trigger value (STI101)')

    plt.show()  # show all plots together

    # Bids preparation
    bids_path = BIDSPath(subject=subject, session=session,
                          task=task, run=run, root=bids_root)
    

    write_raw_bids(raw, bids_path, events_data=events, 
                       event_id=events_id, overwrite=True)
    
    # Write in Maxfilter files
    write_meg_calibration(calibration_file, bids_path=bids_path, verbose=False)
    write_meg_crosstalk(crosstalk_file, bids_path=bids_path, verbose=False)

In [ ]:
# MRI bids conversion
""" using the trans file created by coregistration, we now convert T1W MRI file 
to bids format"""
bids_path = BIDSPath(subject=subject, session=session,
                     task=task, run=run, root=bids_root)
MRI_data_root = r'Z:\Projects\FLUX\MRI_data'  # RDS folder for trans file
trans_fname = op.join(MRI_data_root, 'trans.fif')
fs_sub_dir = r'Z:\Projects\mTBI_predict\Collected_Data\MRI_data\sub-04'  # FreeSurfer directory for this subject #needs modifications
fs_sub = 'sub-04'
t1_fname = op.join(fs_sub_dir, fs_sub + '.nii')

# Create the BIDSpath object
""" creat MRI specific bidspath object and then use trans file to transform 
landmarks from the raw file to the voxel space of the image"""

t1w_bids_path = BIDSPath(subject=subject, session=session, 
                         root=bids_root, suffix='T1w')

info = read_raw_bids(bids_path=bids_path, verbose=False).info
trans = mne.read_trans(trans_fname)  
landmarks = get_anat_landmarks(
    image=t1_fname,  # path to the nifti file
    info=info,  # MEG data file info from the subject
    trans=trans,
    fs_subject=fs_sub,
    fs_subjects_dir=fs_sub_dir)

t1w_bids_path = write_anat(
    image=t1_fname, bids_path=t1w_bids_path,
    landmarks=landmarks, deface=True,
    overwrite=True, verbose=True)